## Import des librairies 

In [ ]:
import pandas as pd 
import math
import numpy as np 
import seaborn as sns 
import plotly.express as px
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from pickle import dump

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Chargement des données 

In [ ]:
data = pd.read_csv("../data/raw/ATP_tweaked.csv", sep=";")
data.head();

In [ ]:
sample = data.head()
sample = sample.drop(columns=['p1_won'])
sample.to_csv("../data/test/sample.csv", index=False)

In [ ]:
print(f"Le jeu de données contient {data.shape[0]} lignes et {data.shape[1]} colonnes.")

In [ ]:
fig = px.pie(data, names='p1_won', title='Proportion of winners and losers')
fig.write_image("../visualizations/proportion_of_winners_and_losers.png")

**Concernant l'équilibre du jeu de données, on a autant de 1 et de 0 pour la variable cible. Dans le cas contraire, les valeurs de p1 et p2 sont interchangeables. On n'aurait pas eu besoin de faire du sur ou sous-échantillonnage pour équilibrer le dataset.**

Avant de commencer à explorer les données et à les pré-traiter, il faut d'abord comprendre les variables avec lesquelles on travaille. On commence par identifier les variables numériques et les variables catégorielles.  

In [ ]:
data.dtypes;

In [ ]:
object_columns = data.select_dtypes(['object']).columns
data[object_columns] = data[object_columns].astype('category')
data[["p1_rank", "p2_rank"]] = data[["p1_rank", "p2_rank"]].astype('category')

Ensuite, on essaie de les explorer grâce à la data visualisation pour en sortir des insights intéréssants pour la phase de modélisation : 

In [ ]:
fig = px.histogram(data, x='p1_rank', color="p1_won", title='Distribution of the rankings of the players')
fig.write_image("../visualizations/distribution_of_the_rankings_of_the_players.png")

In [ ]:
fig = px.histogram(data, x='tourney_level', pattern_shape="p1_won", title="Histogram of tourney levels")
fig.write_image("../visualizations/histogram_of_tourney_levels.png")

In [ ]:
fig = px.histogram(data, x='p1_ioc', color= "p1_won", pattern_shape="p1_won", title="Histogram of players' countries")
fig.write_image("../visualizations/histogram_of_players_countries.png")

Le but du projet est de prédire le gagnant d'un match de tennis. On peut faire la prédiction avant ou pendant le match mais dans ce cas, l'accès à certaines données peut ou non être possible. 

Si on suppose que l'on cherche à prédire le gagnant du match avant le début du match, certaines données tels que la durée du match, le pourcentage de sets gagnants ou le nombre d'avantages etc ne sont pas disponibles. Les variables correspondantes à ces données ne sont donc pas pertinentes pour l'entraînement de notre modèle. 

On considère que ce sera le but de notre projet. Il est donc nécessaire de supprimer l'ensemble de ces variables avant de commencer le pré-traitement des variables restantes. 

In [ ]:
unrelevant_columns = ['score', 'p1_df', 'p2_df', 'p1_bpFaced', 'p2_bpFaced', 'p1_bpSaved', 'p2_bpSaved',
                      'p1_svpt', 'p2_svpt', 'p1_1stIn', 'p2_1stIn', 'p1_1stWon', 'p2_1stWon', 'p1_SvGms',
                      'p2_SvGms', 'p1_2ndWon', 'p2_2ndWon', 'p1_ace', 'p2_ace', 'best_of', 'minutes']

In [ ]:
data = data.drop(columns=unrelevant_columns)
data.head();

## Gestion des valeurs manquantes 

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_data = pd.DataFrame({'column_name': data.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_data;

Les colonnes avec plus de 80% de valeurs manquantes vont être supprimées.

In [ ]:
missing_data_higher_than_80 = missing_value_data[missing_value_data.percent_missing > 80]
missing_data_higher_than_80

In [ ]:
columns_with_too_many_missing_values = missing_data_higher_than_80['column_name']
columns_with_too_many_missing_values

In [ ]:
data = data.drop(columns = columns_with_too_many_missing_values)
data.shape

In [ ]:
data.head();

Pour le reste des colonnes restantes, il faut adopter une autre stratégie pour gérer les valeurs manquantes. Pour les variables numériques, le plus simple est de remplacer les valeurs manquantes par les valeurs moyennes et pour les variables catégorielles, le plus simple est de remplacer par la valeur la plus récurrente : 

In [ ]:
numeric_columns = data.select_dtypes(['int64', 'float64']).columns

In [ ]:
numeric_columns

Les variables numériques `rank_points` dépendent des variables catégorielles `rank`. Par définition, les variables `seed` peuvent ne pas être définis pour tous les joueurs. On va traiter ces variables plus tard. 

In [ ]:
percent_missing_numeric = data[numeric_columns].isnull().sum() * 100 / len(data[numeric_columns])
missing_value_numeric_data = pd.DataFrame({'column_name': numeric_columns,
                                 'percent_missing': percent_missing_numeric})

In [ ]:
missing_value_numeric_data[missing_value_numeric_data.percent_missing > 0];

In [ ]:
data['p1_age'] = data['p1_age'].fillna(data['p1_age'].mean())
data['p2_age'] = data['p2_age'].fillna(data['p2_age'].mean())
data['p1_ht'] = data['p1_ht'].fillna(data['p1_ht'].mean())
data['p2_ht'] = data['p2_ht'].fillna(data['p2_ht'].mean())

In [ ]:
categorical_columns = data.select_dtypes(['category']).columns
percent_missing_categorical = data[categorical_columns].isnull().sum() * 100 / len(data[categorical_columns])
missing_value_categorical_data = pd.DataFrame({'column_name': categorical_columns,
                                 'percent_missing': percent_missing_categorical})

In [ ]:
missing_value_categorical_data[missing_value_categorical_data.percent_missing > 0];

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data[['p1_hand']])
data['p1_hand'] = imputer.transform(data[['p1_hand']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data[['p2_hand']])
data['p2_hand'] = imputer.transform(data[['p2_hand']])

In [ ]:
data.p1_hand.unique()

Les variables `p1_hand` et `p2_hand` prennent également 'U' comme valeur. C'est une valeur aberrante. Toutes les valeurs U seront donc remplacées par R.

In [ ]:
data.loc[data.p1_hand == 'U', 'p1_hand'] = 'R'
data.loc[data.p2_hand == 'U', 'p2_hand'] = 'R'
data['p1_hand'] = data['p1_hand'].astype('category')
data['p2_hand'] = data['p2_hand'].astype('category')

In [ ]:
columns_with_too_many_categories = []
for column in categorical_columns: 
    if data[column].nunique() > 100:
        columns_with_too_many_categories.append(column)

Les variables catégorielles avec beaucoup de catégories vont être supprimées. 

In [ ]:
columns_with_too_many_categories

In [ ]:
columns_to_keep = ['tourney_name', 'p1_rank', 'p2_rank']
columns_with_too_many_categories = [e for e in columns_with_too_many_categories if e not in columns_to_keep]

In [ ]:
data = data.drop(columns=columns_with_too_many_categories)

In [ ]:
data.head();

## Feature Engineering

In [ ]:
data[["p1_id", "p2_id"]] = data[["p1_id", "p2_id"]].astype('category')

In [ ]:
def extract_month(x):
    return int(str(x)[4:6])

In [ ]:
data['tourney_month'] = data['tourney_date'].apply(lambda x: extract_month(x))
data.head();

In [ ]:
data = data.drop(columns=['tourney_date'])

In [ ]:
def is_seed_player(x):
    if math.isnan(x):
        return 0
    return 1

In [ ]:
data['p1_is_seed_player'] = data['p1_seed'].apply(lambda x: is_seed_player(x))

In [ ]:
data['p2_is_seed_player'] = data['p2_seed'].apply(lambda x: is_seed_player(x))

In [ ]:
data.head();

In [ ]:
data = data.drop(columns=['p1_seed', 'p2_seed'])

In [ ]:
def define_rank(x):
    if x < 31:
        return 'Top 30'
    elif x < 101:
        return 'Top 30-100'
    else :
        return 'Under 100'

In [ ]:
data['p1_new_rank'] = data['p1_rank'].apply(lambda x: define_rank(x))
data['p2_new_rank'] = data['p2_rank'].apply(lambda x: define_rank(x))

In [ ]:
data = data.drop(columns=['p1_rank', 'p2_rank'])

In [ ]:
fig = px.histogram(data, x='p2_new_rank')
fig.write_image("../visualizations/distribution_of_the_players_2_type_of_rankings.png")

In [ ]:
fig = px.histogram(data, x='p1_new_rank')
fig.write_image("../visualizations/distribution_of_the_players_1_type_of_rankings.png")

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data[['p1_new_rank']])
data['p1_new_rank'] = imputer.transform(data[['p1_new_rank']])
data['p1_new_rank'] = data['p1_new_rank'].astype('category')

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data[['p2_new_rank']])
data['p2_new_rank'] = imputer.transform(data[['p2_new_rank']])
data['p2_new_rank'] = data['p2_new_rank'].astype('category')

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data[data.p1_new_rank=='Top 30'][['p1_rank_points']])
data.loc[data.p1_new_rank=='Top 30','p1_rank_points'] = imputer.transform(data[data.p1_new_rank=='Top 30'][['p1_rank_points']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data[data.p2_new_rank=='Top 30'][['p2_rank_points']])
data.loc[data.p2_new_rank=='Top 30','p2_rank_points'] = imputer.transform(data[data.p2_new_rank=='Top 30'][['p2_rank_points']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data[data.p1_new_rank=='Top 30-100'][['p1_rank_points']])
data.loc[data.p1_new_rank=='Top 30-100','p1_rank_points'] = imputer.transform(data[data.p1_new_rank=='Top 30-100'][['p1_rank_points']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data[data.p2_new_rank=='Top 30-100'][['p2_rank_points']])
data.loc[data.p2_new_rank=='Top 30-100','p2_rank_points'] = imputer.transform(data[data.p2_new_rank=='Top 30-100'][['p2_rank_points']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data[data.p1_new_rank=='Under 100'][['p1_rank_points']])
data.loc[data.p1_new_rank=='Under 100','p1_rank_points'] = imputer.transform(data[data.p1_new_rank=='Under 100'][['p1_rank_points']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data[data.p2_new_rank=='Under 100'][['p2_rank_points']])
data.loc[data.p2_new_rank=='Under 100','p2_rank_points'] = imputer.transform(data[data.p2_new_rank=='Under 100'][['p2_rank_points']])

In [ ]:
data.isnull().sum() * 100 / len(data);

In [ ]:
corrM = data.corr()
fig = px.imshow(corrM, text_auto=True, aspect="auto")
fig.write_image("../visualizations/correlation_matrix.png")

## Standardisation des variables numériques

On standardise les variables numériques : 

In [ ]:
data.dtypes;

In [ ]:
X = data.drop(columns=['p1_won'])
numeric_columns = X.select_dtypes(['int64', 'float64']).columns

In [ ]:
numeric_columns

In [ ]:
scaler = StandardScaler()
scaler.fit(data[numeric_columns])

In [ ]:
data[numeric_columns] = scaler.transform(data[numeric_columns])

In [ ]:
data.head();

In [ ]:
dump(scaler, open('../models/scaler.pkl', 'wb'))

## Labelisation des variables catégorielles pour les modèles ne les gérant pas par défaut

In [ ]:
data.dtypes;

In [ ]:
final_categorical_columns = data.select_dtypes(['category']).columns

In [ ]:
data_with_encoded_categories = data.copy()

In [ ]:
for column in final_categorical_columns:
    encoder = LabelEncoder()
    encoder.fit(data_with_encoded_categories[column])
    data_with_encoded_categories[column] = encoder.transform(data_with_encoded_categories[column])
    dump(scaler, open(f'../models/encoder-{column}.pkl', 'wb'))

In [ ]:
data_with_encoded_categories.head();

## Enregistrement des données pré-traitées

In [ ]:
data.to_csv("../data/processed/data_preprocessed.csv", index=False)

In [ ]:
data_with_encoded_categories.to_csv("../data/processed/data_with_encoded_categories_preprocessed.csv", index=False)

In [ ]:
data.columns